In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.graph_objects as go
import pandas as pd


def make_fig(df):
    df['구매일'] = pd.to_datetime(df['구매일'])
    df['구매일(월)'] = df['구매일'].dt.strftime("%Y%m")

    df_merge_event = df[(df['물품대분류'] == '체험팩') | 
                                    (df['상품명'].str.startswith('[1+1]')) | 
                                    (df['상품명'].str.startswith('(핫딜위크/1+1)')) | 
                                    (df['상품명'].str.startswith('(1+1)')) | 
                                    (df['상품명'].str.contains('쿠폰적용가')) | 
                                    (df['상품명'].str.contains('증정')) | 
                                    (df['상품명'].str.contains('체험팩'))]

    df_merge_event_new = df_merge_event.groupby('구매일(월)').count()
    df_merge_event_new


    fig = go.Figure(layout=go.Layout(title='이벤트제품 판매량'))
    fig.update_layout(
        title=dict(
            x=0.5,
            font=dict(size=24)
        )
    )

    fig.add_trace(go.Scatter(x= df_merge_event_new.index, 
                            y= df_merge_event_new["성별"],
                            name = '일반',
                            mode='lines+markers+text',
                            line={'color':'#1E90FF',
                                'width':6,
                                },
                            marker={'color':'#666699'},
                            showlegend=True))
    return fig
    
def make_page_event_sale(df):
    fig = make_fig(df)
    
    return [    
        html.H1(children='이벤트제품 판매량'),
        html.Div(children='''
            유아용품 쇼핑몰 A사
        '''),
        dcc.Graph(
            id='graph_event_sale',
            figure=fig
        )
    ]

    
if __name__ == '__main__':
    import import_ipynb
    from jupyter_dash import JupyterDash
    from util.google import url_to_dataframe
    
    processed_df_link = "https://drive.google.com/file/d/1l9S5UU5f7L-OeKWhMRctCdSvVVxlFt_8/view?usp=sharing"

    ############################

    df = url_to_dataframe(processed_df_link)
    df['구매일'] = pd.to_datetime(df['구매일'])
    
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

    app = JupyterDash("6조 대쉬보드", external_stylesheets=external_stylesheets)
    
    app.layout = html.Div(make_page_event_sale(df))
    app.run_server(debug=True)
    